### Sentiment Analysis
#### Using Chatgpt
- Testing on Test Set based on Prompt
#### Using Few Shot
- Takes train set as an input for previous examples
- 

In [6]:
! py -m pip install requirements.txt

Defaulting to user installation because normal site-packages is not writeable
HINT: You are attempting to install a package literally named "requirements.txt" (which cannot exist). Consider using the '-r' flag to install the packages listed in requirements.txt


ERROR: Could not find a version that satisfies the requirement requirements.txt (from versions: none)
ERROR: No matching distribution found for requirements.txt

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
! py -m pip install openai

import openai
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
# import tiktoken
import warnings
import json
from sklearn.model_selection import train_test_split
from sklearn import metrics


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Configure OpenAI connection
openai.api_key = dbutils.secrets.get(scope="openai", key="api-key")
openai.api_base = "https://gsk-ds.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_type = "azure"

NameError: name 'dbutils' is not defined

In [ ]:
class OpenaiAnnotater():
  # openai.api_key = dbutils.secrets.get(scope="openai", key="api-key")
  def __init__(self, engine = "ds-gpt-35", temp = 0):
    self.engine = engine
    self.temp = temp

  def question_answer(self, system_prompt, user_prompt):
    # for prompt in pronmpts:
    messages = [{"role": "system", "content": system_prompt},{"role" : "user", "content" : user_prompt}]
    response = openai.ChatCompletion.create(
      engine = self.engine,
      temperature = self.temp,
      messages=messages,
      max_tokens=2000
    )
    return response.choices[0].message["content"]
  

  def get_completion(self, user_prompt):
    # for prompt in pronmpts:
    messages = [{"role" : "user", "content" : user_prompt}]
    response = openai.ChatCompletion.create(
      engine = self.engine,
      temperature = self.temp,
      messages=messages,
      max_tokens=2000
    )
    return response.choices[0].message["content"]
  

In [ ]:
annotater  = OpenaiAnnotater()

In [ ]:
blob_path = '/dbfs/mnt/medicalaffairs/Deepti_test/Sentiment Analysis//'
reportname='RSV-workbook.xlsx'

In [ ]:
!pip install xlsxwriter
# Specify the Excel file name
out_excel_file_name = blob_path+"RSV_Sentiment_Sample_Out.xlsx"
# Create an ExcelWriter object


writer = pd.ExcelWriter('tmp_out.xlsx', engine='xlsxwriter') #to write in excel - temporary


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
#samples for training
df=pd.read_excel(blob_path+reportname, sheet_name= "Sheet1")

In [ ]:
df.columns

Index(['Unnamed: 0', 'text', 'inputs', 'prediction', 'prediction_agent',
       'annotation', 'annotation_agent', 'vectors', 'multi_label',
       'explanation',
       ...
       'risk_groups_diabetes: 534', 'sentiment_negative: 314',
       'dis_educ_cont_rec_pos: 558', 'dis_educ_cont_rec_neg: 560',
       'risk_groups_sleep_apnea: 565', 'driver_aware: 561',
       'dis_educ_cont_rec_neutral: 559', 'sentiment_neutral: 315',
       'adjuvant_unaware: 466', 'dis_educ_later: 436'],
      dtype='object', length=149)

In [ ]:
df_new= df[(df['safety_favorable: 703']==1) | (df['safety_concerned: 701']==1) | (df['safety_neutral: 702'] == 1)]

In [ ]:
len(df_new)

127

In [ ]:
df_new_1 = df_new[['text','safety_favorable: 703','safety_concerned: 701','safety_neutral: 702']]
df_new_1

,text,safety_favorable: 703,safety_concerned: 701,safety_neutral: 702
10,Arexvy Vaccine Efficacy and Safety Season 2\nI...,1,0,0
16,Coadministration project\nP4 pharmacy student ...,0,0,1
75,CME is pulmonologist and MUN provider in large...,0,0,1
89,Questions asked at the presentation: re the RS...,0,0,1
118,"No Differences in Efficacy, But Concerned for ...",0,1,0
...,...,...,...,...
2495,Arexvy VE and Safety\nSchool of Pharmacy Pharm...,1,0,0
2508,Arexvy coadmin with Fluad\nClinical Services c...,0,1,0
2540,ASO1b\nEE said it was a good thing we didn't u...,0,1,0
2566,Vaccine safety\nEE is a pharmacy manager of an...,0,1,0


In [ ]:
df_train, df_test = train_test_split(df_new_1, random_state=32, test_size=0.30, shuffle=True)

print(df_train.shape) #this will be required for few shots only
print(df_test.shape)

(88, 4)
(39, 4)


In [ ]:
df_test.head(2)

,text,safety_favorable: 703,safety_concerned: 701,safety_neutral: 702
482,What data is important for community immunizer...,1,0,0
2097,Clear Difference in Efficacy \nEE is family me...,1,0,0


### Using Chatgpt to predict sentiment on test class: df_test - no other input needed

In [ ]:
sentiment_list= ['favorable', 'neutral', 'concerned']
business_context="""To understand HCP's perception of safety profile of GSK's RSV vaccine - this will help drive decision making (I.e. education)"""
rationale="""To understand if HCP inquire about or mention the safety of the rsv vaccine? Does HCP speak favorably of GSK's RSV vaccine. Example: "HCP not concerned about safety profile - adverse events were as expected". Is the HCP concerned about the safety of GSK's RSV vaccine? HCP may show concerns about the safety GSK's RSV vaccine. This could include: concerns about adverse events or any other hesitations with regards to safety. Does HCP indicates that he/ she is neutral towards GSK's RSV vaccine. Examples: "if safety is good then patients will get the vaccine"; "more information on reactogenicity"""
def get_sentiment_output(text_sentiment):

  prompt = f"""
  Act as an AI medical expert, an automated service to identify if the sentiment in a mvoc_stmt is from \
  the list of sentiments mentioned in sentiment_list, separated by comma. Analyze the sentiment of mvoc_stmt while considering the business context and rationale to gain a comprehensive understanding given in business_context and rationale in next line\
  Businesscontext: ```{business_context}```\

  Rationale: ```{rationale}```\
  
  Sentiment List: ```{sentiment_list} ```\
  
  After predicting the sentiment match it with the list given in sentiment_list. \
  Output the matched sentiment with header 'predicted_sentiment'.   \
  The statement is made in the USA and delimited by triple backticks as follows: 
  
  mVOC: ```{text_sentiment}```. 

  Be precise. Do not add any information which is not present in the data. Cross-check your answer before for accuracy and logical consitency. Donot output the result in commas or inverted commas" 

  """

 
  response=annotater.get_completion(prompt)
  #print(response)
  return response

In [ ]:
df_chatgpt_response= pd.DataFrame()
for index, row in df_test.iterrows():

  response= get_sentiment_output(row['text'])
  #print(response)
  df_chatgpt_response.at[index,'Text']= row['text']
  df_chatgpt_response.at[index,'Predicted Sentiment']=response

  


predicted_sentiment: neutral
predicted_sentiment: favorable
predicted_sentiment: neutral
predicted_sentiment: 'concerned'
predicted_sentiment: neutral
predicted_sentiment: neutral
predicted_sentiment: 'favorable'
predicted_sentiment: neutral
predicted_sentiment: 'favorable'
predicted_sentiment: 'favorable'
predicted_sentiment: 'concerned'
predicted_sentiment: 'concerned'
predicted_sentiment: 'favorable'
predicted_sentiment: 'concerned'
predicted_sentiment: 'favorable'
predicted_sentiment: neutral
predicted_sentiment: 'neutral'
predicted_sentiment: neutral
predicted_sentiment: 'neutral'
predicted_sentiment: 'concerned'
predicted_sentiment: 'neutral'
predicted_sentiment: 'concerned'
predicted_sentiment: 'favorable'
predicted_sentiment: neutral
predicted_sentiment: 'favorable'
predicted_sentiment: neutral
predicted_sentiment: neutral
predicted_sentiment: 'favorable'
predicted_sentiment: neutral
predicted_sentiment: favorable
predicted_sentiment: favorable
predicted_sentiment: neutral
pred

In [ ]:

df_chatgpt_response['Predicted Sentiment'] = df_chatgpt_response['Predicted Sentiment'].str.replace('predicted_sentiment: ', "")
df_chatgpt_response['Predicted Sentiment'] = df_chatgpt_response['Predicted Sentiment'].str.replace("'", "")
df_chatgpt_response.head(2)

#map_to_nos = {'favorable':0, 'neutral':1, 'concerned':2}

#df_chatgpt_response['Predicted Sentiment'] = df_chatgpt_response['Predicted Sentiment'].map(map_to_nos)


,Text,Predicted Sentiment
482,What data is important for community immunizer...,neutral
2097,Clear Difference in Efficacy \nEE is family me...,favorable


In [ ]:
len(df_chatgpt_response)

39

###Metric

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
len(df_test)

39

In [ ]:
#Actual values
df_test1 = df_test.rename(columns={'safety_favorable: 703': 'favorable', 'safety_neutral: 702': 'neutral', 'safety_concerned: 701' : 'concerned' })
print(len(df_test1))
'''
melted_df= df_test1.melt(id_vars=["text"], 
        var_name="Actual Sentiment"
        )

'''
for index, row in df_test1.iterrows():
  if row['favorable']==1:
      df_test1.at[index, 'Actual Sentiment'] = 'favorable'
  elif row['neutral']==1:
      df_test1.at[index, 'Actual Sentiment'] = 'neutral'
  elif row['concerned']==1:
      df_test1.at[index, 'Actual Sentiment'] = 'concerned'
  else:
    pass 




39


In [ ]:
confusion_matrix(df_test1['Actual Sentiment'].tolist(), df_chatgpt_response['Predicted Sentiment'].tolist())
confusion_matrix

<function sklearn.metrics._classification.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)>

In [ ]:


print(metrics.classification_report(df_test1['Actual Sentiment'].tolist(), df_chatgpt_response['Predicted Sentiment'].tolist()))

              precision    recall  f1-score   support

   concerned       1.00      0.50      0.67        14
   favorable       0.64      0.53      0.58        17
     neutral       0.33      0.75      0.46         8

    accuracy                           0.56        39
   macro avg       0.66      0.59      0.57        39
weighted avg       0.71      0.56      0.59        39



##USING FEW SHOT APPROACH
#Favorable, concerned neutral all 3 at the same time

#########Filter mvocs that are diverse in df_train_for_chatgpt to use as few examples for this approach

In [ ]:
df_train.to_csv(blob_path+'filtered_rsv_sentiments.csv')

In [ ]:

df_fews_examples_manual_checked =pd.read_excel(blob_path+'filtered_rsv_sentiments_manual_verified.xlsx', sheet_name="rsv_text_manuall_filtered") #case1#filtered_rsv_sentiments
#df_fews_examples_manual_checked =pd.read_excel(blob_path+'filtered_rsv_sentiments_manual_verified.xlsx', sheet_name="concerned_neutral_favorable_eq ") #howvere, this even lowers the accuracy because low number of mvocs to learn from

from sklearn.utils import shuffle
df_fews_examples_manual_checked = shuffle(df_fews_examples_manual_checked)

In [ ]:
system_prompt= f"""Act as a medical expert in USA. Identify the sentiment in a mvoc_stmt based on the examples provided in the next message and the output should only be from the list of sentiments mentioned in sentiment_list, separated by comma. 
  
  Sentiment List: ```{sentiment_list} ```\

  """

for index, row in df_fews_examples_manual_checked.iloc[-40:].iterrows(): # 
#for index, row in df_fews_examples_manual_checked.iloc[-30:].iterrows(): #considering equal number tabbed sheets
  text= row['text']
  sent=""
  if row['safety_favorable: 703']==1:
      sent = 'favorable'
  elif row['safety_neutral: 702']==1:
      sent = 'neutral'
  elif row['safety_concerned: 701']==1:
      sent = 'concerned'

  
  prompt = f""" mVOC{index} : {text} \
                Sentiment: {sent}\
              """
              

  system_prompt += f"{prompt}"

system_prompt += "Be precise. Do not add any information which is not present in the data. Cross-check your answer before for accuracy and logical consitency"      

In [ ]:
df_fewshot_response= pd.DataFrame()
for index, row in df_test.iterrows():

  user_prompt= f""" 
                    mVOC: {row['text']} \
                    Sentiment: <SENTIMENT>



              """

  response= annotater.question_answer(system_prompt, user_prompt)

  df_fewshot_response.at[index,'Text']= row['text']
  df_fewshot_response.at[index,'Predicted Sentiment']=response

In [ ]:

df_fewshot_response['Predicted Sentiment'] = df_fewshot_response['Predicted Sentiment'].str.replace('Sentiment: ', "")
df_fewshot_response.head(5)

In [ ]:

actual_list=df_test1['Actual Sentiment'].tolist()
predicted_list=df_fewshot_response['Predicted Sentiment'].tolist()
actual_list = [item.lower() for  item in actual_list]
predicted_list = [item.lower() for item in predicted_list]


print(metrics.classification_report(actual_list, predicted_list))

####Extract summary of train samples to use for few shot one at a time

In [ ]:
df_fews_examples_manual_checked

In [4]:
! py -m pip freeze > requirements.txt